In [1]:
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datetime import datetime
from google.colab import drive
import os, zipfile, glob
import pickle
import re
import gensim

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install unidecode

     |████████████████████████████████| 235 kB 8.6 MB/s 


In [8]:
import codecs
from bs4 import BeautifulSoup
import unidecode
import random
import re

In [9]:
!pip install spacy

In [10]:
!pip install contractions

     |████████████████████████████████| 284 kB 6.5 MB/s 
     |████████████████████████████████| 321 kB 62.6 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85438 sha256=1ddd3141f7eb00da6cce43cfe48c365cda40ffc97b90c8f626dc855de35c24af
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [11]:
import contractions

In [12]:
def decontracted(phrase):

    phrase = re.sub(r'[^\w.?!;]', ' ', phrase)
    phrase = re.sub(' +', ' ', phrase)
    
    #remove html tags from text
    soup = BeautifulSoup(phrase, "html.parser")
    phrase = soup.get_text(separator=" ")
    
    #remove accented characters from text, e.g. café
    phrase = unidecode.unidecode(phrase)
    
    
    
    sentences = re.split('([.;!?] *)', phrase)

    return ' '.join([i.capitalize() for i in  sentences])

In [13]:
# Modules for languages
# en_core_web_sm - for English
# it_core_news_sm - for Italian
# es_core_news_sm - for Spanish


!python -m spacy download es_core_news_sm

     |████████████████████████████████| 16.2 MB 8.4 MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-py3-none-any.whl size=16172933 sha256=01df0315d7eab035655a43e7f245d588fb03959a4235e95f5453bb8ff7fa509e
  Stored in directory: /tmp/pip-ephem-wheel-cache-4h_wckw8/wheels/21/8d/a9/6c1a2809c55dd22cd9644ae503a52ba6206b04aa57ba83a3d8
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [14]:
import es_core_news_sm

In [15]:
# initializing the language model

nlp = es_core_news_sm.load(disable=['parser'])
nlp.max_length = 10000

In [21]:
def prepare_english_text(input_path, output_path, error_list, nlp, enc = 'utf-8', gutenberg = True, title = ''):
    
    #English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer.
    
    #parser: The dependency parser jointly learns sentence segmentation and labelled dependency parsing, 
    #and can optionally learn to merge tokens that had been over-segmented by the tokenizer.
    
    pos_dict = {'PROPN': 'PERSON1', 'PRON': 'PRON1', 'NUM': 'ORDINAL1'}

    
    try:
        raw_text = codecs.open(input_path, 'r', enc).read()
    except:
        return -1
    
    if gutenberg:
        begin = raw_text.find('*** START OF THIS PROJECT GUTENBERG EBOOK ')
        begin_2 = begin + 42 + len(title) + 3
        if begin == -1:
                begin = raw_text.find('START OF THE PROJECT GUTENBERG EBOOK ')
                begin_2 = begin + 37 + len(title) + 3

        end = raw_text.find('*** END OF THIS PROJECT GUTENBERG EBOOK')
        if end == -1:
            end = raw_text.find('*** END')
        raw_text = raw_text[begin_2 : end]
        
    
    preprocessed_text = decontracted(raw_text)
    
    nlp_doc = nlp(preprocessed_text)
    sorted_ents = sorted(nlp_doc.ents, key = lambda x: len(x), reverse =  True)


    for ent in sorted_ents:
        preprocessed_text = preprocessed_text.replace(' ' + ent.text + ' ', ' ' + ent.label_+ '1 ')
        
        if not ent.text.islower():
            preprocessed_text = preprocessed_text.replace(' ' + ent.text.lower() + ' ', ' ' + ent.label_+ '1 ')

    new_nlp_doc = nlp(preprocessed_text)
    file_name = path.split('/')[-1]
    

    with open(output_path + file_name, 'w+', ) as prepared_text:
        for token in new_nlp_doc:
            if token.text[-1] != '1':
                if token.pos_ in pos_dict:
                    try:
                        prepared_text.write(pos_dict[token.pos_])
                    except:
                        error_list.append(token.pos_)
                    prepared_text.write('\n')
                    
                        
                    
                elif token.pos_ != 'PUNCT':
                    try:
                        prepared_text.write(token.lemma_.lower())
                    except:
                        error_list.append(token.pos_)
                        
                    prepared_text.write('\n')
                    

            else:
                try:
                    repared_text.write(token.text)
                except:
                        error_list.append(token.pos_)
                prepared_text.write('\n')
    return 1

In [17]:
ind_gen = list(glob.glob("/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/*.txt"))

In [18]:
ind_gen[:10]

['/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14000.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14001.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14002.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14003.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14004.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14005.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14006.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14007.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14008.txt',
 '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/doc_14009.txt']

In [22]:
# Let's process only big files, 
# because there are a lot of small files, which is why processing takes a very long time

tocken_error = []
prepare_book = []
num = 0
book_err = []
sum_len = 0

for path in ind_gen:
    if num % 100 == 0:
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        print(dt_string, '| ',  'number of processed files: ' + str(num))
    
    title = path.split('/')[-1].split('.txt')[0]
    
    res = prepare_english_text(path, '/content/drive/MyDrive/2022-01-15_Course_project/prep_get_text_es/', 
                               tocken_error, nlp, gutenberg = False, title = title)
    if res < 0:
      book_err.append(path)
      print('ERROR: ', path)
    else:
        prepare_book.append(path)
    num += 1

18/01/2022 22:48:05 |  number of processed files: 0
18/01/2022 22:48:56 |  number of processed files: 100
18/01/2022 22:49:47 |  number of processed files: 200
18/01/2022 22:50:37 |  number of processed files: 300
18/01/2022 22:51:26 |  number of processed files: 400
18/01/2022 22:52:17 |  number of processed files: 500
18/01/2022 22:53:06 |  number of processed files: 600
18/01/2022 22:53:56 |  number of processed files: 700
18/01/2022 22:54:47 |  number of processed files: 800
18/01/2022 22:55:38 |  number of processed files: 900
18/01/2022 22:56:29 |  number of processed files: 1000
18/01/2022 22:57:20 |  number of processed files: 1100
18/01/2022 22:58:11 |  number of processed files: 1200
18/01/2022 22:59:02 |  number of processed files: 1300
18/01/2022 22:59:53 |  number of processed files: 1400
18/01/2022 23:00:45 |  number of processed files: 1500
18/01/2022 23:01:38 |  number of processed files: 1600
18/01/2022 23:02:31 |  number of processed files: 1700
18/01/2022 23:03:22 | 